# Adversarial Training

__Objective__: Mitigate the model with **targeted attacks** using **Cleverhans** attack library.

__Workflow__:
1.  Load the `Traffic Sign` dataset and examine its structure.
    - Download `tsrd-train` from http://www.nlpr.ia.ac.cn/pal/trafficdata/recognition.html.
    - Take a closer look at the data inspecting it's size, shape, and quantity.
    - View some random samples using either OpenCV or Matplotlib (or others) of the handwritten digits and observe the complexity of the images. Do you have any issues identify these written numbers?
2. Using `numpy` or other libraries prepare your dataset for training
    - Ensure the format or shape of the data is appropriate for input into your model (one-hot-encoding is needed for the labels)
    - Ensure data types are correct and that data is normalized. 
3. Load and convert the model that is trained in the previous project
   - load using Keras' `load_model` method
   - convert it to logit model using `tf.keras.Model` with `model.input` as the input parameter and `model.layers[-1].output` as the output parameter.
4. Generate malicious inputs
   - FGSM attack
      - Generate malicious inputs based on X_train and FGSM attack using CleverHans' `fast_gradient_method` class with 0.08 epsilon value
5. Adversarial training
   - Create a loop. In each iteration; 
      - generate new malicious inputs for the DL model.
      - Merge the generated malicious with the training dataset
      - Retraind the model using `model.fit` method with the new training dataset
   - Save the new defended model
6. Select first 8 images from test dataset, Plot random 24 images (e.g. 3 rows and 8 columns).
   - First row shows the original images with the predicted labels
   - Second row shows the FGSM based manipulated images with the predicted labels
   - Third row shows the `random uniform` manipulated images with the predicted labels
7. Generate malicious inputs based on X_train and FGSM attack using CleverHans' `fast_gradient_method` class with 0.08 epsilon value with target class `16`

__Deliverable__:

The deliverable is a Jupyter Notebook documenting your workflow. You should save `trainX,testX,trainY,testY,predictions,x_test_adv_fgsm,y_test_adv_fgsm,y_adv_pred_fgsm_hat` numpy matrices as you'll need it for further analysis in the second milestone.

In [1]:
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent

from keras.models import load_model
from art.estimators.classification import KerasClassifier
import tensorflow as tf
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import cv2
import os
from keras.preprocessing.image import  img_to_array
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import EarlyStopping

np.random.seed(10)

In [2]:
# Step 1
IMAGE_PATH_TRAIN = '../../data/tsrd-train/'
COL_NAMES = ['filename','f1','f2','x1','y1','x2','y2','label']
IMAGE_SIZE = (134,128)
EPOCHS = 30
BS = 100
MODEL_NAME = 'cnn-traffic-sign.h5'

In [3]:
# Step 2
df = pd.read_csv('../../data/TsignRecgTrain4170Annotation.txt',names=COL_NAMES,sep=';', index_col=False)

files = glob(IMAGE_PATH_TRAIN + '*.png')
data = []
labels = []

for fname in tqdm(range(len(files)),):
    fname = files[fname]
    image = cv2.imread(fname)
    image = cv2.resize(image, IMAGE_SIZE)
    image = img_to_array(image)
    data.append(image)
    
    label = df.query("filename=='" + os.path.split(fname)[1] + "'").label.values[0]
    labels.append(label)
    
data = np.array(data) / 255.0
labels = to_categorical(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

  0%|          | 0/4170 [00:00<?, ?it/s]

In [4]:
# Step 3
MODEL_NAME = '../../Part_1/chinese_traffic_sign_classifier_v1.h5'
model = load_model(MODEL_NAME)
# Create the logit model from tf.keras.Model
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

2021-12-09 11:12:19.008968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 11:12:19.014489: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 11:12:19.014772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 11:12:19.015297: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
predictions = model.predict(testX)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(testY, axis=1)) / len(testY)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

2021-12-09 11:12:29.612827: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-09 11:12:29.949853: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-12-09 11:12:30.193771: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Accuracy on benign test examples: 26.258992805755394%


## 1 Attack results on undefended DL model

In [6]:
# Step 4: Generate adversarial test examples with **targeted Projected Gradient Descent** attack
ATTACK_POWER = 0.1
NUM_OF_MALICIOUS_INPUTS = 150
TARGET_CLASS = 15

randIdx = np.random.randint(0,testX.shape[0],NUM_OF_MALICIOUS_INPUTS)

y_attack_target = (np.ones((NUM_OF_MALICIOUS_INPUTS,)) * TARGET_CLASS).astype(int)

x_test_adv_pgd = projected_gradient_descent(logits_model, testX[randIdx,:],
                                            eps=ATTACK_POWER, norm=np.inf, nb_iter=400, 
                                            eps_iter=ATTACK_POWER/10.0, targeted=True,
                                            y=y_attack_target)

2021-12-09 11:13:16.848944: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.38GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-12-09 11:13:17.415008: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.42GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


### Initial attack success ratio
Here, we will evaluate the **undefended** DL model for the PGD based malicious inputs.

In [7]:
# Evaluate the the classifier on adversarial test examples
y_adv_pred_pgd = model.predict(x_test_adv_pgd)
attack_success_ratio = np.sum(np.argmax(y_adv_pred_pgd, axis=1) == TARGET_CLASS) / NUM_OF_MALICIOUS_INPUTS
print("Attack success ratio on first {} adversarial test samples: {}%".format(NUM_OF_MALICIOUS_INPUTS, attack_success_ratio*100))

Attack success ratio on first 150 adversarial test samples: 64.66666666666666%


## 2 Adversarial Training
We will try to defen the DL model for PGD based malicious inputs with target class 15. The number of iteration is 10.

In [10]:
# Step 5: Adversarial training
NUM_OF_RETRAIN_ITER = 10

y_test_adv_pgd = testY[randIdx,:]
print(y_test_adv_pgd.shape)

es = EarlyStopping(monitor='val_loss', 
                   patience=5, 
                   min_delta=0.001,
                   verbose=1,
                   restore_best_weights=True,
                   mode='min')

for i in tqdm(range(NUM_OF_RETRAIN_ITER)):
    logits_model = tf.keras.Model(model.input, model.layers[-1].output)
    x_test_adv_pgd = projected_gradient_descent(logits_model, testX[randIdx,:],
                                                eps=ATTACK_POWER, norm=np.inf, nb_iter=100, 
                                                eps_iter=ATTACK_POWER/10.0, targeted=True,
                                                y=y_attack_target)
    
    x_train_tmp = np.concatenate((trainX, x_test_adv_pgd), axis=0)
    y_train_tmp = np.concatenate((trainY, y_test_adv_pgd), axis=0)
    
    model.fit(x_train_tmp, y_train_tmp, epochs=100,verbose=0,callbacks=[es],
              validation_data=(x_test_adv_pgd, y_test_adv_pgd), batch_size=16)
model.save('cnn-traffic-sign-defended-for-pgd.h5')

(150, 58)


  0%|          | 0/10 [00:00<?, ?it/s]

Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping


2021-12-09 11:18:35.471125: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.85MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-12-09 11:18:45.474346: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 880.0KiB (rounded to 901120)requested by op gradient_tape/sequential/max_pooling2d_1/MaxPool/MaxPoolGrad
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2021-12-09 11:18:45.474420: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2021-12-09 11:18:45.474459: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 54, Chunks in us

ResourceExhaustedError:  OOM when allocating tensor with shape[16,32,20,22] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/sequential/max_pooling2d_1/MaxPool/MaxPoolGrad (defined at tmp/ipykernel_17697/793370899.py:24) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_8497]

Function call stack:
train_function


### Attack success ratio on Defended model
Now, we will evaluate the **defended** DL model for the PGD based malicious inputs. The result would be lower than attack success ratio on **undefended** DL model.

In [ ]:
# Evaluate the the classifier on adversarial test examples
x_test_adv_pgd = projected_gradient_descent(logits_model, testX[randIdx,:],
                                            eps=ATTACK_POWER, norm=np.inf, nb_iter=100, 
                                            eps_iter=ATTACK_POWER/10.0, targeted=True,
                                            y=y_attack_target)
y_adv_pred_pgd = model.predict(x_test_adv_pgd)
attack_success_ratio = np.sum(np.argmax(y_adv_pred_pgd, axis=1) == TARGET_CLASS) / NUM_OF_MALICIOUS_INPUTS
print("Attack success ratio on first {} adversarial test samples: {}%".format(NUM_OF_MALICIOUS_INPUTS, attack_success_ratio*100))

In [ ]:
# Step 6: Plot original, noisy and malicious inputs
NUM_OF_ROWS = 3
NUM_OF_COLS = 8
fig, ax = plt.subplots(NUM_OF_ROWS,NUM_OF_COLS, figsize=(15,6))

for i in range(NUM_OF_COLS):
    uniform_noise = np.random.uniform(low=-1.0,high=1.0,size=testX[i].shape) * ATTACK_POWER
    noise = testX[i:i+1] + uniform_noise
    
    benign_pred = model.predict(testX[i:i+1])
    malicious_pred = model.predict(x_test_adv_pgd[i:i+1])
    noise_pred = model.predict(noise)

    ax[0,i].imshow(testX[i])
    ax[1,i].imshow(x_test_adv_pgd[i])
    ax[2,i].imshow(noise[0].clip(0.0,1.0))

    ax[0,i].set_title('Label:' + str(benign_pred.argmax(axis=1)))
    ax[1,i].set_title('Label:' + str(malicious_pred.argmax(axis=1)))
    ax[2,i].set_title('Label:' + str(noise_pred.argmax(axis=1)))
    ax[0,i].axis('off')
    ax[1,i].axis('off')
    ax[2,i].axis('off')
plt.show()

### Attack success ratio on Defended model with another target
Now, we will evaluate the **defended** DL model for the PGD based malicious inputs for another target class (e.g. 16). The result would be higher than attack success ratio on **previous** target class. To protect the DL model for other target class based attacks, you need to perform adversarial training for the other class also.

In [ ]:
# Step 7: Attack success ratio on Defended model with another target
NEW_TARGET_CLASS = 16
y_attack_target_new = (np.ones((NUM_OF_MALICIOUS_INPUTS,)) * NEW_TARGET_CLASS).astype(int)

x_test_adv_pgd = projected_gradient_descent(logits_model, testX[randIdx,:],
                                            eps=ATTACK_POWER, norm=np.inf, nb_iter=100, 
                                            eps_iter=ATTACK_POWER/10.0, targeted=True,
                                            y=y_attack_target_new)

y_adv_pred_pgd = model.predict(x_test_adv_pgd)
attack_success_ratio = np.sum(np.argmax(y_adv_pred_pgd, axis=1) == NEW_TARGET_CLASS) / NUM_OF_MALICIOUS_INPUTS
print("Attack success ratio on first {} adversarial test samples: {}%".format(NUM_OF_MALICIOUS_INPUTS, attack_success_ratio*100))

In [ ]:
# Save matrices
np.savez('milestone-4-1.npz', trainX=trainX, testX=testX,
         trainY=trainY, testY=testY, predictions=predictions,
         x_test_adv_pgd=x_test_adv_pgd,
         y_test_adv_pgd=y_test_adv_pgd)